# Imports #

In [33]:
import pandas as pd
#from openai import OpenAI
from collections.abc import MutableMapping
import json
#!pip install openai
from openai import OpenAI
import os
import openai

import pandas as pd

from PIL import Image
#!pip install opencv-python
from PIL import Image
import collections
import math




In [ ]:
# testing if image opens properly... 
# Imports PIL module  
from PIL import Image 
  
# open method used to open different extension image file 
im = Image.open(data["filepath_og_img"][1])  
  
# This method will show image in any image viewer  
im.show()  

# Cleaning the CloudResearch Data 

In [7]:
#Reading response data
output_file_path = "1k_CloudResearch_study/1k_dataset_output.csv"
data = pd.read_csv(output_file_path)

#Filling additional info description box with string
data.fillna("No description provided", inplace = True)

# create the relative file_path of the image to access w/ DALLE
data["Filepath_og_img"] = data["Filename_og_img"].apply(lambda file: "1k-MTurk-Images/" + file)

In [44]:
test_data = data.iloc[1:3]

In [45]:
test_data

,age_description,jaw_description,eyes_description,hair_description,nose_description,race_description,beard_description,gender_description,eyebrows_description,expression_description,...,Sex,Occupation Field,Relationship/Marital Status,Political Party,Gender,Country Of Residence,Household Income,Race,Employment Status,Filepath_og_img
1,mid 30s,"pointed, soft","small, brown, friendly","short, brown, straight","wide, small nostrils",white,"brown, scruffy, mustash",male,"brown, whispy",smiling,...,Female,Education & Training,Married,Independent,Woman,United States,"$100,000-$124,999",White,Full-time,1k-MTurk-Images/Google_1_Louis Sung_1_oval.jpg
2,90's,a round jaw,light blue eyes,short cropped hair,wide nose,black,no beard,female,thinning light brows,smiling,...,Female,Retail,In a relationship,Prefer not to say,Woman,United States,Prefer not to say,White,Full-time,1k-MTurk-Images/Google_1_Lorraine Byrd_9_oval.jpg


# Creating DALLE Images

In [ ]:
# create a new folder for the target directory that contains the DALLE images if it does not exist
target_folder = "DALLE_images" 
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, target_folder)
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

In [5]:
test_data

,age_description,jaw_description,eyes_description,hair_description,nose_description,race_description,beard_description,gender_description,eyebrows_description,expression_description,additionalInfo_description,Attractive_og_img,Race_og_img,Memorable_og_img,Filename_og_img,AWSFile_og_img,Task Data,Submitted Data
0,40s,Rounded,Hazel,Brown short,Smaller sharp,white,None,female,Brown,smiling,No description provided,4.0,1.0,4.0,Google_1_Jane Standifer_11_oval.jpg,https://mturk-1k-images.s3.amazonaws.com/Googl...,"{""RowData"":[{""CellData"":""4.0"",""ColumnHeader"":""...","{""Data"":{""taskData"":{""age"":""40s"",""jaw"":""Rounde..."
1,mid 30s,"pointed, soft","small, brown, friendly","short, brown, straight","wide, small nostrils",white,"brown, scruffy, mustash",male,"brown, whispy",smiling,"not weraling anything else, flat ears",4.0,1.0,5.0,Google_1_Louis Sung_1_oval.jpg,https://mturk-1k-images.s3.amazonaws.com/Googl...,"{""RowData"":[{""CellData"":""4.0"",""ColumnHeader"":""...","{""Data"":{""taskData"":{""age"":""mid 30s"",""jaw"":""po..."
2,90's,a round jaw,light blue eyes,short cropped hair,wide nose,black,no beard,female,thinning light brows,smiling,wearing earrings and smiling,3.0,2.0,4.0,Google_1_Lorraine Byrd_9_oval.jpg,https://mturk-1k-images.s3.amazonaws.com/Googl...,"{""RowData"":[{""CellData"":""3.0"",""ColumnHeader"":""...","{""Data"":{""taskData"":{""age"":""90\u0027s"",""jaw"":""..."
3,42,Roundish,Hazel,Gray,Average,white,Slight gray,male,Gray,neutral,Seemed to be wea,3.0,1.0,3.0,Adam_Koester_6_oval.jpg,https://mturk-1k-images.s3.amazonaws.com/Adam_...,"{""RowData"":[{""CellData"":""3.0"",""ColumnHeader"":""...","{""Data"":{""taskData"":{""age"":""42"",""jaw"":""Roundis..."
4,37,Wide,Brown,Brown,Narrow,white,Shaved,male,Narrow,smiling,No description provided,3.0,5.0,3.0,Google_1_Joe Amaral_3_oval.jpg,https://mturk-1k-images.s3.amazonaws.com/Googl...,"{""RowData"":[{""CellData"":""3.0"",""ColumnHeader"":""...","{""Data"":{""taskData"":{""age"":""37"",""jaw"":""Wide"",""..."


In [13]:
testing = test_data.iloc[0]
testing_name = testing["Filename_og_img"]
testing_name.split(".jpg")[0] + "_DALLE.jpg"

'Google_1_Jane Standifer_11_oval_DALLE.jpg'

In [98]:
import requests
from PIL import Image
from io import BytesIO

def convert_to_jpg(url, output_path):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image.save(output_path, 'JPEG')
            print(f"Image saved as {output_path}")
            return output_path
        else:
            print("Failed to download image from URL.")
    except Exception as e:
        print(f"A error occurred: {e}")
        return "error"


In [99]:
# create a new folder for the target directory that contains the images
target_folder = "DALLE_images"
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, target_folder)
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

In [23]:
path = "1k_CloudResearch_study/1k_survey_2.csv"
file_name = path.split("/")[1]
new_file_name = file_name.split(".jpg")[0] + "_DALLE.jpg"
output_path = "DALLE_images/" + new_file_name
output_path

'DALLE_images/1k_survey_2.csv_DALLE.jpg'

In [11]:
data.iloc[3]

age_description                                                               42
jaw_description                                                         Roundish
eyes_description                                                           Hazel
hair_description                                                            Gray
nose_description                                                         Average
race_description                                                           white
beard_description                                                    Slight gray
gender_description                                                          male
eyebrows_description                                                        Gray
expression_description                                                   neutral
additionalInfo_description                                      Seemed to be wea
Attractive_og_img                                                            3.0
Race_og_img                 

In [112]:
def generate_and_save_DALLE_image(row):
    prompt = ("Photo of a human face by photographer Anna-Lou Leibovitz against a white backdrop, with a camera angle of 80mm at eye level, while also containing the follwing features: " 
              +  row['gender_description'] + "; Age: " + row['age_description'] + "; Race: " 
              + row['race_description'] + "; Hair: " + row['hair_description'] + "; Eyebrows: " 
              + row['eyebrows_description'] + "; Eyes: " + row["eyes_description"] + "; Nose: " 
              + row["nose_description"] + "; Jaw: " +  row['jaw_description'] + "; Expression: " + row["expression_description"] +
    "; Beard: " + row['beard_description'] + "; Additional Info: " + row['additionalInfo_description'])

    OPENAI_API_KEY = "key"

    client = OpenAI(api_key = OPENAI_API_KEY)

    try: 
        # generate image for each user prompt
        response = client.images.generate(
            model="dall-e-2",
            prompt= prompt,
            size="256x256",
            quality="standard",
            n=1,
        )
        
        url = response.data[0].url
    except openai.RateLimitError as error:
        # Handle the RateLimitError here
        print("Rate limit exceeded:", error)
    except: 
        print("a content error occurred")
        return("content error")

    # converting DALLE image to jpg and saving to DALLE_images folder 
    file_name = row["Filepath_og_img"]
    file_name = file_name.split("/")[1]
    new_file_name = file_name.split(".jpg")[0] + "_DALLE.jpg"
    output_path = "DALLE_images/" + new_file_name
    DALLE_file_path = convert_to_jpg(url, output_path)

    return DALLE_file_path 

In [105]:
# need to figure out how to time this so it makes each request properly 
#data["DALLE_image_path"] = data.apply(lambda row: generate_and_save_DALLE_image(row), axis = 1)

In [113]:
import time

def create_DALLE_images(df):
    """
    Applies the generate_and_save_DALLE_image fx to stay w/in the openai rate limit 
    """
    # Initialize variables for rate limiting
    start_time = time.time()
    image_count = 0
    DALLE_file_paths = []
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        DALLE_file_path = generate_and_save_DALLE_image(row)
        DALLE_file_paths.append(DALLE_file_path)
        # Apply DALL-E function to the current row
        # Replace the placeholder function call with your DALL-E function call
        # dalle_output = your_dalle_function(row['column_name'])

        # Simulate DALL-E function call with a sleep of 1 second
        time.sleep(1)

        # Update image count and check rate limit
        image_count += 1
        if image_count >= 5:
            # If rate limit exceeded, wait until the next minute
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            # Reset image count and start time
            image_count = 0
            start_time = time.time()

        # Print progress
        print(f"Processed {index + 1} rows")
    
    # creating a new column in the df with the corresponding file paths
    df["DALLE_image_path"] = DALLE_file_paths
    print("DALLE image processing completed.")
    return df


In [109]:
# need to figure out how to time this so it makes each request properly 
test_data = create_DALLE_images(test_data)

a content error occurred
Processed 2 rows
Image saved as DALLE_images/Google_1_Lorraine Byrd_9_oval_DALLE.jpg
Processed 3 rows
DALLE image processing completed.


/var/folders/wz/7k6v05bx5gvffw4vkwfkmkn40000gn/T/ipykernel_66122/2010554336.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DALLE_image_path"] = DALLE_file_paths


# Standardizing DALLE Images

### methods to crop images and add oval to match the og image

In [84]:
from PIL import Image
import math

def crop_image(og_image_file, DALLE_image_file):
    """
    Crops the DALLE image according to the dimensions of the original image 
    Args: 
        og_image: filename of original image
        DALLE_image: filename of DALLE image

    Returns: 
        Path of new cropped DALLE image file 
    """

    # Open the original image
    og_image = Image.open(og_image_file)
    width_og, height_og = og_image.size

    # Open the DALLE image
    DALLE_image = Image.open(DALLE_image_file)
    width, height = DALLE_image.size

    # Calculate the dimensions for cropping
    width_diff = width - width_og
    height_diff = height - height_og
    width_diff_one_side = width_diff / 2
    height_diff_one_side = height_diff / 2

    # Ensure the image is larger than the original image
    if width_diff < 0 or height_diff < 0:
        print("Error: Cropped area is larger than the original image.")
        return None

    # Calculate the cropping coordinates
    left = math.floor(width_diff_one_side)
    right = width - math.floor(width_diff_one_side)
    top = math.floor(height_diff_one_side)
    bottom = height - math.floor(height_diff_one_side)

    # Perform the cropping
    cropped_img = DALLE_image.crop((left, top, right, bottom))

    # Save the cropped image
    file_name = DALLE_image_file.split("/")[-1]  
    path = file_name.split(".")[0]
    transformed_image_path = "TRANSFORMED_DALLE_images/" + path + "_TRANSFORMED.jpg"
    cropped_img.save(transformed_image_path)
    
    return transformed_image_path


In [77]:
def get_image_pixels(image_file):
    # Open the image
    im = Image.open(image_file)

    # Load pixel data
    pix = im.load()

    # Get the width and height of the image
    width, height = im.size

    # Create a 2D array to store pixel values
    pixels_2d = [[0 for _ in range(width)] for _ in range(height)]

    # Iterate over each pixel and store its RGBA values in the 2D array
    for x in range(width):
        for y in range(height):
            pixels_2d[y][x] = pix[x, y]

    return pixels_2d


In [92]:
def create_image_from_arr(image_path, pixels_2d):
    # Assuming pixels_2d is your 2D array of pixels
    # Example pixels_2d = [[(255, 255, 255), (0, 0, 0), ...], [...], ...]

    # Assuming width and height are the dimensions of your image
    width, height = len(pixels_2d[0]), len(pixels_2d)

    # Create a new image
    new_im = Image.new("RGB", (width, height))

    # Load pixel data
    pix = new_im.load()

    # Set pixel values
    for x in range(width):
        for y in range(height):
            pix[x, y] = pixels_2d[y][x]

    # Save the image to a file
    new_im.save(image_path)
    return image_path

In [93]:
def apply_white_oval(og_image_file, AI_image_file):
    """
    Uses breadth-first search to identify the white oval frame in the og image and apply it to the AI generated image
    og_image_file, AI_image_file
    """

    # getting the 2d array of pixels from each image
    og_image_pixels = get_image_pixels(og_image_file)
    AI_image_pixels = get_image_pixels(AI_image_file)

    rows, cols = len(og_image_pixels), len(og_image_pixels[0])
    rows1, cols1 = len(AI_image_pixels), len(AI_image_pixels[1])
    print(rows, cols, rows1, cols1)
    
    visited = set()

    def bfs(r, c):
        q = collections.deque()
        visited.add((r, c))
        q.append((r, c))

        while q:
            row, col = q.popleft()
            directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]

            for dr, dc in directions:
                r, c = row + dr, col + dc
                if (r in range(rows) 
                    and c in range(cols) 
                    and og_image_pixels[r][c] == (255, 255, 255)
                    and (r, c) not in visited):
                    q.append((r, c))
                    visited.add((r, c))
                    AI_image_pixels[r][c] = (255, 255, 255)

    # breadth first search starting from the corners of the image (to capture the oval)
    bfs(0, 0)
    bfs(rows - 1, 0)
    bfs(0, cols - 1)
    bfs(rows - 1, cols - 1)
    output_image_path = create_image_from_arr(AI_image_file, AI_image_pixels)
    return output_image_path

In [94]:
def transform_DALLE_image(og_image_file, DALLE_image_file):
    """
    Standardizes the DALLE image by cropping to fit dimensions of original image and applying the white oval
    """
    cropped_image = crop_image(og_image_file, DALLE_image_file)
    transformed_image_path = apply_white_oval(og_image_file, cropped_image)
    return transformed_image_path

### implementing methods

In [95]:
# create a new folder for the target directory that contains the transformed images if it does not exist
target_folder = "TRANSFORMED_DALLE_images" 
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, target_folder)
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

In [96]:
test_data["DALLE_images_transformed"] = test_data.apply(lambda row: transform_DALLE_image(row["Filepath_og_img"], row["DALLE_image_path"]) 
                                                        if row["DALLE_image_path"] != "error occurred" 
                                                        else row["DALLE_image_path"], axis = 1)

256 209 256 210


/var/folders/wz/7k6v05bx5gvffw4vkwfkmkn40000gn/T/ipykernel_66122/1939585345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["DALLE_images_transformed"] = test_data.apply(lambda row: transform_DALLE_image(row["Filepath_og_img"], row["DALLE_image_path"])
